In [1]:
import torch
from torch import nn
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
df = pd.read_csv('../data/assignment_1/advertising.csv', sep=',')

In [3]:
df

,TV,Radio,Newspaper,Sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,12.0
3,151.5,41.3,58.5,16.5
4,180.8,10.8,58.4,17.9
...,...,...,...,...
195,38.2,3.7,13.8,7.6
196,94.2,4.9,8.1,14.0
197,177.0,9.3,6.4,14.8
198,283.6,42.0,66.2,25.5


In [32]:
adv_tensor = torch.tensor(df.values, dtype=torch.float32)

In [37]:
X = adv_tensor[:, 0:3]
y = adv_tensor[:, 3]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [39]:
X_train = X_train.requires_grad_(True)
X_test = X_test.requires_grad_(True)
y_train = y_train.requires_grad_(True)
y_test = y_test.requires_grad_(True)

In [40]:
class RegressionModel(nn.Module):
	def __init__(self, input_dim, output_dim):
		super().__init__()
		self.linear = nn.Linear(input_dim, output_dim, dtype=torch.float32)
		
	def forward(self, x):
		return self.linear(x)

In [41]:
epochs = 1000
learning_rate = 0.01
model = RegressionModel(3, 1)

In [42]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

losses = []

In [43]:
for epoch in range(epochs):
    model.train()

    y_pred = model(X_train).squeeze()
    
    train_loss = loss_fn(y_pred, y_train)
    
    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    model.eval()

    with torch.no_grad():
        y_pred_test = model(X_test).squeeze()
        test_loss = loss_fn(y_pred_test, y_test)

    losses.append([train_loss.item(), test_loss.item()])
    if (epoch+1) % 50 == 0:
        print(f'Epoch: {epoch+1}/{epochs}, Train Loss: {train_loss.item():.4f}, Test Loss: {test_loss.item():.4f}')

Epoch: 50/1000, Train Loss: 237.8044, Test Loss: 292.9208
Epoch: 100/1000, Train Loss: 92.4938, Test Loss: 124.6982
Epoch: 150/1000, Train Loss: 40.9519, Test Loss: 65.7446
Epoch: 200/1000, Train Loss: 21.8319, Test Loss: 39.3960
Epoch: 250/1000, Train Loss: 13.2206, Test Loss: 24.5531
Epoch: 300/1000, Train Loss: 8.8518, Test Loss: 15.7732
Epoch: 350/1000, Train Loss: 6.7018, Test Loss: 10.8712
Epoch: 400/1000, Train Loss: 5.7151, Test Loss: 8.2699
Epoch: 450/1000, Train Loss: 5.2837, Test Loss: 6.9140
Epoch: 500/1000, Train Loss: 5.0910, Test Loss: 6.1964
Epoch: 550/1000, Train Loss: 4.9905, Test Loss: 5.7994
Epoch: 600/1000, Train Loss: 4.9217, Test Loss: 5.5638
Epoch: 650/1000, Train Loss: 4.8622, Test Loss: 5.4103
Epoch: 700/1000, Train Loss: 4.8045, Test Loss: 5.2987
Epoch: 750/1000, Train Loss: 4.7464, Test Loss: 5.2080
Epoch: 800/1000, Train Loss: 4.6874, Test Loss: 5.1275
Epoch: 850/1000, Train Loss: 4.6275, Test Loss: 5.0515
Epoch: 900/1000, Train Loss: 4.5669, Test Loss: 4.9

In [25]:
print(model.get_parameter('linear.weight'))
print(model.get_parameter('linear.bias'))

Parameter containing:
tensor([[0.0649, 0.1380, 0.0306]], requires_grad=True)
Parameter containing:
tensor([1.0018], requires_grad=True)
